In [437]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,chi2,SequentialFeatureSelector as sfs

In [438]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [439]:
## setting work directory

In [440]:
os.chdir("U:\Propensity_Model\Model")

In [441]:
### Loading the required files

In [442]:
pickle_in = open("Final_Data_.pickle","rb")

Final_Data = pickle.load(pickle_in)

pickle_in.close()

In [443]:
### Split data into Train, Test and Validation

In [444]:
x = Final_Data.drop('AUDIT_STATUS',axis=1)
y = pd.DataFrame(Final_Data['AUDIT_STATUS'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [445]:
## Checking the Selection %

In [446]:
print(y_train['AUDIT_STATUS'].value_counts()/len(y_train)*100) #1.77
print(y_test['AUDIT_STATUS'].value_counts()/len(y_test)*100)   #1.88

0    98.226685
1     1.773315
Name: AUDIT_STATUS, dtype: float64
0    98.119926
1     1.880074
Name: AUDIT_STATUS, dtype: float64


In [447]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [448]:
train_num = x_train.select_dtypes(include='number')

In [449]:
##PreProcessing numeric data

In [450]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [451]:
### Combining the processed data to train_data

In [452]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [453]:
## Combining the PreProcessed data to train_data

In [454]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [455]:
x_train = x_train.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [456]:
## performing one-hot-encoding on categorical features

In [457]:
ctgrl_col = pd.DataFrame(x_train[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

In [458]:
### Data balancing using SMOTE

In [459]:
sm = SMOTE(sampling_strategy=0.43)

In [460]:
x_sm,y_sm = sm.fit_resample(x_train.values,y_train.values.ravel())

In [461]:
x_sm = pd.DataFrame(x_sm)
y_sm = pd.DataFrame(y_sm)

In [462]:
x_sm.columns = x_train.columns
y_sm.columns = y_train.columns

In [463]:
y_sm.value_counts(normalize=True)*100

AUDIT_STATUS
0               69.931045
1               30.068955
dtype: float64

In [464]:
y_train.value_counts(normalize=True)*100

AUDIT_STATUS
0               98.226685
1                1.773315
dtype: float64

In [465]:
####################################################
####################################################
####################################################

### Model Building

In [466]:
NB_v1 = GaussianNB()

In [467]:
data = {
    'var_smoothing': np.logspace(0,-9,num=100)
}

In [468]:
from sklearn.model_selection import GridSearchCV
gscv = GridSearchCV(NB_v1,data,n_jobs=-1,verbose=True,scoring='accuracy')
gscv.fit(x_sm.values,y_sm.values.ravel())

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             scoring='accuracy', verbose=True)

In [469]:
gscv.best_params_

{'var_smoothing': 0.002848035868435802}

In [470]:
gscv.best_score_

0.6228263801564836

In [471]:
NB_v1 = GaussianNB(var_smoothing=0.002848035868435802)
NB_v1.fit(x_sm.values,y_sm.values.ravel())
#sfs1 = sfs(RF_v1,n_features_to_select=20,direction='backward',scoring='roc_auc',n_jobs=-1)

GaussianNB(var_smoothing=0.002848035868435802)

In [472]:
#sfs1.fit(x_sm,y_sm)

In [473]:
#sfs1.support_

In [474]:
#sfs1.feature_names_in_

In [475]:
#x_sm = pd.DataFrame(x_sm[['REV_0300','REV_0250']])

In [476]:
#RF_v1 = RandomForestClassifier(n_estimators=200,min_samples_split=7,min_samples_leaf=5,max_features='sqrt',max_depth=30,criterion='entropy')
#RF_v1.fit(x_sm.values,y_sm.values.ravel())

In [477]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [478]:
test_num = x_test.select_dtypes(include='number')

In [479]:
##PreProcessing numeric data

In [480]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [481]:
###########
### Combining the processed data to test_data

In [482]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [483]:
## Combining the Preprocessed data to test_data

In [484]:
x_test = pd.concat([x_test,test_num_process],axis=1)

In [485]:
x_test = x_test.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [486]:
##one-hot-encoding on categorical features

In [487]:
ctgrl_col = pd.DataFrame(x_test[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)

In [488]:
#x_test = pd.DataFrame(x_test[['REV_0300','REV_0250']])

In [489]:
##################
### Predicting on the test data

In [490]:
##RF_v1

In [497]:
y_pred = NB_v1.predict(x_test)
y_pred = (NB_v1.predict_proba(x_test)[:,1]>0.5).astype('int').astype('str')
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


0.5012106537530266
[[3415 3474]
 [  28  104]]
              precision    recall  f1-score   support

           0       0.99      0.50      0.66      6889
           1       0.03      0.79      0.06       132

    accuracy                           0.50      7021
   macro avg       0.51      0.64      0.36      7021
weighted avg       0.97      0.50      0.65      7021



In [492]:
y_pred = NB_v1.predict(x_sm)
y_pred = (NB_v1.predict_proba(x_sm)[:,1]>0.5).astype('int').astype('str')
print(accuracy_score(y_sm,y_pred))
print(confusion_matrix(y_sm,y_pred))

print(classification_report(y_sm,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


0.6211529686153222
[[14006 13579]
 [ 1365 10496]]
              precision    recall  f1-score   support

           0       0.91      0.51      0.65     27585
           1       0.44      0.88      0.58     11861

    accuracy                           0.62     39446
   macro avg       0.67      0.70      0.62     39446
weighted avg       0.77      0.62      0.63     39446

